In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


In [ ]:
raw_data,info = tfds.load('cats_vs_dogs',with_info=True,split=['train[:80%]','train[80%:100%]'])
print(info)
train_data,test_data = raw_data

In [ ]:
def process_map(data):

    img = tf.cast(data['image'],dtype=tf.float32)/255.0
    img = tf.image.resize(img,(100,100))

    label = data['label']
    return img,label


train_data = train_data.shuffle(1000)
train_data = train_data.map(process_map)

for ele in train_data.take(1):
    img = ele[0]
    show = plt.imshow(img)
    plt.title( "this is a {}".format(ele[1]) )
    plt.show()

train_data = train_data.batch(128)

In [ ]:
initializer = tf.keras.initializers.he_normal( seed=2 )

def ResBlock(Ain,block_num,f_num):

    # conv ops
    # reconnection after 2 layer

    l1 = 'block_{}_layer_1'.format(block_num)
    l2 = 'block_{}_layer_2'.format(block_num)

    out = tf.keras.layers.Conv2D(f_num,(3,3),padding='SAME',activation='relu',kernel_initializer=initializer,kernel_constraint=tf.keras.constraints.MinMaxNorm(
              min_value=-2.0, max_value=2.0, rate=1.0, axis=[0, 1, 2]
          ),name=l1)(Ain)
    out = tf.keras.layers.Conv2D(f_num,(3,3),padding='SAME',kernel_initializer=initializer,kernel_constraint=tf.keras.constraints.MinMaxNorm(
              min_value=-2.0, max_value=2.0, rate=1.0, axis=[0, 1, 2]
          ),name=l2)(out)

    return out


def ResNetfunc(x_batch,num_of_blocks):

    out = tf.keras.layers.BatchNormalization()(x_batch)
    out = tf.keras.layers.Conv2D(32,(3,3),activation='relu',kernel_initializer=initializer,kernel_constraint=tf.keras.constraints.MinMaxNorm(
              min_value=-2.0, max_value=2.0, rate=1.0, axis=[0, 1, 2]
          ))(out)
    out = tf.keras.layers.MaxPool2D((2,2))(out)

    
    block_out = ResBlock(out,1,32)
    out = block_out + out
    out = tf.keras.activations.relu(out)

    
    block_out = ResBlock(out,2,32)
    out = block_out + out
    out = tf.keras.activations.relu(out)

    out = tf.keras.layers.MaxPool2D((2,2))(out)

    out = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',kernel_initializer=initializer,kernel_constraint=tf.keras.constraints.MinMaxNorm(
              min_value=-2.0, max_value=2.0, rate=1.0, axis=[0, 1, 2]
          ))(out)
    out = tf.keras.layers.BatchNormalization()(out)

    for i in range(3):
      block_out = ResBlock(out,i+3,64)
      out = block_out + out
      out = tf.keras.activations.relu(out)

      if i==1:
        out = tf.keras.layers.MaxPool2D((2,2))(out)

    out = tf.keras.layers.MaxPool2D((2,2))(out)

    return out

def DenseNet(res_out):

    out = tf.keras.layers.Flatten()(res_out)
    out = tf.keras.layers.Dense(32,activation='relu')(out)
    out = tf.keras.layers.Dense(16,activation='relu')(out)
    out = tf.keras.layers.Dense(1,activation='sigmoid')(out)

    return out

inputs = tf.keras.layers.Input(shape=(100,100,3),name="image_input")
res_out = ResNetfunc(inputs,5)
outputs = DenseNet(res_out)
model = tf.keras.Model(inputs=inputs,outputs=outputs)
print( model.summary() )

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.05)
# cost_func = tf.keras.losses.BinaryCrossentropy()

def cost_func(y_batch,pred):

    y_batch = tf.cast( tf.expand_dims(y_batch,-1) ,tf.float32)
    pred = tf.expand_dims(pred,-1) 
    
    t1 = tf.linalg.matmul( tf.transpose(y_batch) , tf.math.log(pred) )
    t2 = tf.linalg.matmul( tf.transpose(1-y_batch) , tf.math.log(1-pred) )
    cost = -(t1+t2)
    return tf.squeeze(cost)

def single_step(x_batch,y_batch):

    with tf.GradientTape() as tape:
      pred = model(x_batch)
      pred = tf.squeeze(pred)
      cost = cost_func(y_batch,pred)
    
    grad = tape.gradient(cost,model.trainable_variables)
    optimizer.apply_gradients( zip(grad,model.trainable_variables) )

    return pred,grad,cost

In [ ]:
checkpoint_path = "training_resnet/cp-{epoch:04d}.ckpt"

In [ ]:
def train_net(dataset,epochs):

  loss_mean = tf.keras.metrics.Mean()

  for e in range( epochs ):

    loss_mean.reset_states()
    ctr = 0

    for x_batch,y_batch in iter(dataset):

      pred,grad,cost = single_step(x_batch,y_batch)
      loss_mean.update_state(cost)
      # break

      if ctr%50==0:
          print("Step-Loss   ===>  {}".format( loss_mean.result() ))

      ctr = ctr +1

    print("Epoch :{}:   ----Loss   ===>  {}".format(e, loss_mean.result() ))
    model.save_weights(checkpoint_path.format(epoch=e))

  return   

In [ ]:
with tf.device('/device:GPU:0'):
    train_net(train_data,1),
    model.save('resmodel.h5')

In [ ]:
model.save('resmodel.h5')
from google.colab import files
files.download('resmodel.h5')

In [ ]:
test_data = test_data.map(process_map)
test_data = test_data.batch(64)

In [ ]:
acc = tf.keras.metrics.Accuracy()
mean_acc = tf.keras.metrics.Mean()

for x_batch,y_batch in iter(test_data):

    pred = model(x_batch)
    pred_final = tf.squeeze( tf.cast(pred>0.5,tf.int8) )

    _ = acc.update_state(y_batch,pred_final)
    mean_acc.update_state(acc.result())

print("Final Mean Accuracy is ===>  {}".format(mean_acc.result()))
